In [1]:
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
import random
from diffusers import DDIMScheduler
import numpy as np
from Freeprompt.diffuser_utils import FreePromptPipeline
from Freeprompt.freeprompt_utils import register_attention_control_new
from torchvision.utils import save_image
from torchvision.io import read_image
from Freeprompt.freeprompt import SelfAttentionControlEdit,AttentionStore


#### Model Construction

In [2]:
# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_path = "runwayml/stable-diffusion-v1-5"

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = FreePromptPipeline.from_pretrained(model_path, scheduler=scheduler).to(device)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/home/novikova/opt/izar1/cross/lib/python3.10/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:106: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.14.0",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "set_alpha_to_one": false,
  "steps_offset": 0,
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull request for the `scheduler/scheduler_config.json` file
  deprecate("steps_offset!=1", "1.0.0", deprecation_message, standard_warn=False)


In [6]:
prompts_all = [

    #     ["Photo of a black leather skirt on a light background",
    # "Photo of a red leather skirt on a light background"],
    
    # ["Image of sleek navy blue denim jeans on a wooden surface",
    # "Image of sleek navy blue silk jeans on a wooden surface"],
    
    # ["Photo of an elegant silk evening gown in deep burgundy, draped on a hanger",
    # "Photo of an elegant silk evening gown in deep green, draped on a hanger"],
    
    # ["Snapshot of a floral sundress in sunlight",
    # "Snapshot of a striped sundress in sunlight"],
    
    # ["Photo of a pink cotton t-shirt on grass",
    # "Photo of a black cotton t-shirt on grass"],

    # ["Photo of a yellow cotton dress on white table",
    # "Photo of a pink cotton dress on white table"],
    
    # ["Photo of a white linen button-up shirt on grass",
    # "Photo of a blue linen button-up shirt on grass"],
    
    ["Photo of a pleated midi skirt in pastel pink on white table",
    "Photo of a tight mini skirt in pastel pink on white table"],
    
    # ["Photo of a red cotton midi skirt on grass",
    # "Photo of a green cotton midi skirt on grass"],
    
    # ["Snapshot of classic black denim jeans on a wooden surface",
    # "Snapshot of classic blue denim jeans on a wooden surface"]
]

#### Fake image editing

In [7]:

# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def seed_everything(seed: Optional[int] = None, workers: bool = False) -> int:

    if seed is None:
        seed = os.environ.get("PL_GLOBAL_SEED")
    seed = int(seed)

    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    os.environ["PL_SEED_WORKERS"] = f"{int(workers)}"

    return seed

seed = 42
seed_everything(seed)

self_replace_steps = .8
NUM_DIFFUSION_STEPS = 50

out_dir = "examples/outputs"


for prompts in prompts_all:

    source_prompt = prompts[0]
    target_prompt = prompts[1]
    
    start_code = torch.randn([1, 4, 64, 64], device=device)
    start_code = start_code.expand(2, -1, -1, -1)
    
    latents = torch.randn(start_code.shape, device=device)
    prompts = [source_prompt, target_prompt]
    
    start_code = start_code.expand(len(prompts), -1, -1, -1)
    controller = SelfAttentionControlEdit(prompts, NUM_DIFFUSION_STEPS, self_replace_steps=self_replace_steps)
    
    register_attention_control_new(pipe, controller)
    
    results = pipe(prompts,
                        latents=start_code,
                        guidance_scale=7.5,
                        )
    
    save_image(results[0], os.path.join(out_dir, str(source_prompt)+'.jpg'))
    save_image(results[1], os.path.join(out_dir, str(target_prompt)+'.jpg'))


DDIM Sampler: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:21<00:00,  2.33it/s]
